In [45]:
!pip install transformers


You should consider upgrading via the 'c:\Miniconda\python.exe -m pip install --upgrade pip' command.


In [46]:
import pandas as pd
# because the dataset is int tsv format we have to use delimeter.
df = pd.read_csv("../output/hdfs/HDFS.log_labeled.csv.zip" , names=[ 'blockid','sentence','label'])
df = df.iloc[1:4 , :] # drop first row
# creating a copy so we don't messed up our original dataset.
data=df.copy()
data['label']=data['label'].astype(float)
data['label']=data['label'].astype(int)
data.head(10)


,blockid,sentence,label
1,blk_-1608999687919862906,['Receiving block blk_-1608999687919862906 src...,0
2,blk_7503483334202473044,['Receiving block blk_7503483334202473044 src:...,0
3,blk_-3544583377289625738,['Receiving block blk_-3544583377289625738 src...,1


In [47]:
data.drop(['blockid'],axis=1,inplace=True)
sentences=data.sentence.values
labels = data.label.values
data.head()


,sentence,label
1,['Receiving block blk_-1608999687919862906 src...,0
2,['Receiving block blk_7503483334202473044 src:...,0
3,['Receiving block blk_-3544583377289625738 src...,1


In [ ]:
from transformers import BertTokenizer
# using the low level BERT for our task.
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

# Printing the original sentence.
print(' Original: ', sentences[0])

# Printing the tokenized sentence in form of list.
print('Tokenized: ', tokenizer.tokenize(sentences[0]))

# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences[0])))


In [49]:
input_ids = []
for sent in sentences:
    # so basically encode tokenizing , mapping sentences to thier token ids after adding special tokens.
    encoded_sent = tokenizer.encode(
                        sent,                      # Sentence which are encoding.
                        add_special_tokens = True, # Adding special tokens '[CLS]' and '[SEP]'

                         )
    
 
    input_ids.append(encoded_sent)


Token indices sequence length is longer than the specified maximum sequence length for this model (11928 > 512). Running this sequence through the model will result in indexing errors


In [50]:
from keras.preprocessing.sequence import pad_sequences

MAX_LEN = 128

input_ids = pad_sequences(input_ids, maxlen=MAX_LEN , truncating="post", padding="post")



In [51]:
attention_masks = []

for sent in input_ids:
    
    # Generating attention mask for sentences.
    #   - when there is 0 present as token id we are going to set mask as 0.
    #   - we are going to set mask 1 for all non-zero positive input id.
    att_mask = [int(token_id > 0) for token_id in sent]
    
   
    attention_masks.append(att_mask)


In [52]:
from sklearn.model_selection import train_test_split

train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, test_size=0.2)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, labels,test_size=0.1)


In [53]:
type(validation_labels[0])

numpy.int32

In [57]:
!pip uninstall torch
!pip uninstall torchvision

^C


In [58]:
#conda install pytorch torchvision cudatoolkit=10.1 -c pytorch

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'pytorch'
You should consider upgrading via the 'c:\Miniconda\python.exe -m pip install --upgrade pip' command.


In [59]:
#!pip install torch
#!pip install torchvision 
#import torch
#import torchvision
#changing the numpy arrays into tensors for working on GPU. 
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)

train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)

train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# Deciding the batch size for training.

batch_size = 32

#DataLoader for our training set.
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# DataLoader for our validation(test) set.
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)


C:\Users\jads\AppData\Local\Temp\2\ipykernel_9528\1545515418.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_inputs = torch.tensor(train_inputs)
C:\Users\jads\AppData\Local\Temp\2\ipykernel_9528\1545515418.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  validation_inputs = torch.tensor(validation_inputs)
C:\Users\jads\AppData\Local\Temp\2\ipykernel_9528\1545515418.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_labels = torch.tensor(train_labels)
C:\Users\jads\AppData\Local\Temp\2\ipykernel_9528\1545515418.py:10: U

In [66]:
import torch
torch.cuda.is_available()

False

In [60]:
import time
from transformers import BertForSequenceClassification, AdamW, BertConfig


model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels = 2,   
    output_attentions = False,
    output_hidden_states = False,
)

# Running the model on GPU.
model.cuda()

# Running the model on CPU.
#model.cpu()


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

AssertionError: Torch not compiled with CUDA enabled

In [ ]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, 
                  eps = 1e-8 
                )


In [ ]:
loss_values = []

for epoch_i in range(0, epochs):
    t0 = time.time()

    total_loss = 0
    # putting model in traing mode there are two model eval and train for model
    model.train()
    
    
    for step, batch in enumerate(train_dataloader):
        
        #checking time taken after every 50 steps.
        if step % 50 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
        
        #getting ids,mask,labes for every batch
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        
        outputs = model(b_input_ids, 
                    token_type_ids=None, 
                    attention_mask=b_input_mask, 
                    labels=b_labels)

        loss = outputs[0]

        total_loss += loss.item()
        
        # doing back propagation
        loss.backward()

        optimizer.step()
        
        # Update the learning rate.
        scheduler.step()

    avg_train_loss = total_loss / len(train_dataloader) 
